In [17]:
import csv 
import scipy 
import sklearn
import os
import random
from random import shuffle
import numpy as np
import math
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#libraries for keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
#libraries for plotting
import matplotlib
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
print("successful run")

successful run


In [18]:
counter = 0
labels_age = []
labels_sex = []
cpg_intensities = np.zeros((656,437037))
with open("../../../Datasets/out.csv") as f:
    for row in f:
        participant_all_data = row.split(',')
        labels_age.append(participant_all_data[1])
        labels_sex.append(participant_all_data[0])
        cpg_intensities[counter] = participant_all_data[2:437039]
        counter += 1
labels_age = np.array(labels_age)
labels_sex = np.array(labels_sex)
print(cpg_intensities.shape)
print(labels_age.shape)
print(labels_sex.shape)
print("end")


(656, 437037)
(656,)
(656,)
end


In [19]:
# shuffle the data
import random
import collections
indices = [i for i in range(counter)]
random.shuffle(indices)
shuffled_labels_age = [float(labels_age[i]) for i in indices]
shuffled_labels_sex = [int(float(labels_sex[j])) for j in indices]
shuffled_cpg_intensities = [cpg_intensities[k] for k in indices]

shuffled_labels_age = np.array(shuffled_labels_age)
shuffled_labels_sex = np.array(shuffled_labels_sex)
shuffled_cpg_intensities = np.array(shuffled_cpg_intensities)

print(shuffled_cpg_intensities.shape)
occurrences = collections.Counter(shuffled_labels_sex)
#print(shuffled_labels_sex)



(656, 437037)


In [20]:
print("number of males = ", occurrences[1])
print("number of females = ", occurrences[0])

number of males =  318
number of females =  338


In [21]:
# split into training and test sets
train_size = int(0.8*len(shuffled_labels_age))
print(train_size)
Y_train_age = shuffled_labels_age[:train_size]
Y_test_age = shuffled_labels_age[train_size:len(shuffled_labels_age)]

Y_train_sex = shuffled_labels_sex[:train_size]
Y_test_sex = shuffled_labels_sex[train_size: len(shuffled_labels_sex)]

X_train_cpgs = shuffled_cpg_intensities[:train_size]
X_test_cpgs = shuffled_cpg_intensities[train_size:len(shuffled_cpg_intensities)]

print(X_train_cpgs.shape)
print(X_test_cpgs.shape)

524
(524, 437037)
(132, 437037)


In [22]:
# set model parameters
node_string = '1024,1024,1024,1024'#argv[1]#'256,512,1024',1024,1024,1024'
nodes_size = list(np.array(node_string.split(','),dtype='int32'))
print(nodes_size)
activation_hidden = 'sigmoid'#'relu','sigmoid'
optimizer_use = 'sgd' #'adam','sgd'
validation_split_use = 0.2
epochs_use = 50
batch_size_use = 32
print("successful run")

[1024, 1024, 1024, 1024]
successful run


In [23]:
# create model
model = Sequential()
n_cols = X_train_cpgs.shape[1]

In [24]:
# add model layers
cats = [1,0]
# # layers = len(nodes_size)
for g in range(len(nodes_size)):
    if g == 0:
        model.add(Dense(nodes_size[g], activation = activation_hidden, input_shape=(n_cols,)))
    else:
        model.add(Dense(nodes_size[g], activation = activation_hidden))

model.add(Dense(len(cats), activation='softmax'))
print("successful run")

successful run


In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1024)              447526912 
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 2050      
Total params: 450,677,762
Trainable params: 450,677,762
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compile model using accuracy as a measure of model performance
model.compile(optimizer=optimizer_use, loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)

In [ ]:
# train DNN
Y_train_sex = to_categorical(Y_train_sex)
Y_train_age = Y_train_age # For age
history_callback =model.fit(X_train_cpgs, Y_train_sex, validation_split=validation_split_use, epochs=epochs_use, callbacks=[early_stopping_monitor],batch_size=batch_size_use,verbose=1)
#history_callback =model.fit(X_train_cpgs, Y_train_age, validation_split=validation_split_use, epochs=epochs_use, callbacks=[early_stopping_monitor],batch_size=batch_size_use,verbose=1)



Train on 419 samples, validate on 105 samples
Epoch 1/50
419/419 [==============================] - 297s 708ms/step - loss: 0.7710 - acc: 0.4630 - val_loss: 0.8815 - val_acc: 0.4476
Epoch 2/50
419/419 [==============================] - 221s 527ms/step - loss: 0.7249 - acc: 0.4869 - val_loss: 0.9891 - val_acc: 0.4476
Epoch 3/50
419/419 [==============================] - 293s 699ms/step - loss: 0.7275 - acc: 0.4821 - val_loss: 0.9848 - val_acc: 0.4476
Epoch 4/50
256/419 [=================>............] - ETA: 1:02 - loss: 0.7723 - acc: 0.4922

In [ ]:
#evaluate the model accuracy
_, train_accuracy = model.evaluate(X_train_cpgs, Y_train_sex)
print('Train Accuracy: %.2f' % (train_accuracy*100))

In [ ]:
#test set accuracy
Y_test_sex = to_categorical(Y_test_sex)
_, test_accuracy = model.evaluate(X_test_cpgs, Y_test_sex)
Y_predictions = model.predict(X_test_cpgs)
print(Y_predictions)
print('Test Accuracy: %.2f' % (test_accuracy*100))

In [ ]:
# count number of wrongly classified p = 1/p = 0
counter = 0
misclass_males = 0
misclass_females = 0
print(Y_predictions)
for pred in Y_predictions:
    print(np.where(pred == max(pred)))
    #if np.where(pred == max(pred)) != Y_test_sex[counter]:
    #    if Y_test_sex[counter] == 1:
     #       misclass_males += 1
     #   else:
      #      misclass_females += 1
    counter += 1
print("misclassified males = ", misclass_males)
print("misclassified females = ", misclass_females)

In [ ]:
#plot loss v val_loss for accuracy
loss=np.array(history_callback.history["loss"])
val_loss=np.array(history_callback.history["val_loss"])

ymax=np.max([loss,val_loss])+(np.max([loss,val_loss])*0.1)
ymin=np.min([loss,val_loss])-(np.min([loss,val_loss])*0.1)

x=np.arange(1,len(loss)+1)

legend=['loss','val_loss']

rcParams['figure.figsize'] = 10, 5
plt.title('Accuracy')

plt.xlim(1,len(loss))
plt.ylim(ymin,ymax)
plt.ylabel('loss', fontsize = 16)
plt.xlabel('epoch', fontsize = 16)
plt.plot(x,loss,'-',color='red',label='training loss')
plt.plot(x,val_loss,'-',color='blue',label='validation loss')
plt.legend(loc='upper right')

In [ ]:
plt.show()

In [ ]:
#plot barplot predictions
pops=list(cats)

cmap = matplotlib.cm.get_cmap('rainbow')

cols=[]
for g in range(len(pops)):
    cols.append(cmap(g/(len(pops)-1)))


confusion_mat=np.zeros((len(pops),len(pops)),'int32')

for g in range(len(pops)):
    for gg in range(len(Y_test_cat)):
        if Y_test_cat[gg]==g:
            confusion_mat[g][Y_test_res_cat[gg]]+=1

confusion_mat_per=confusion_mat/np.sum(confusion_mat,axis=1)

rcParams['figure.figsize'] = 20, 10

for g in range(len(pops)):
    #plt.bar(list(np.arange(len(pops))), confusion_mat[:,g], width=0.8, label=pops[g], color=cols[g], bottom=np.sum(confusion_mat[:,g+1:],axis=1))
    plt.bar(list(np.arange(len(pops))), confusion_mat_per[:,g], width=0.8, label=pops[g], color=cols[g], bottom=np.sum(confusion_mat_per[:,g+1:],axis=1))

plt.xticks(list(np.arange(len(pops))), pops)
plt.ylabel("Predicted population")
plt.xlabel("True population")
#plt.legend(loc="upper right")
plt.legend(bbox_to_anchor = (1.01, 1.0))

plt.show()